In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import time

from stabilvol.utility.classes.data_extraction import DataExtractor

In [ ]:
filename = '../data/interim/JT.pickle'
extractor = DataExtractor(start_date='2002-01-01', duration=12, criterion_value=10)
gf = extractor.extract_data(filename)
gf

In [ ]:
total_std = gf.std().mean()
start = -0.1 * total_std
end = -1.5 * total_std
print(total_std, start, end, sep='\n')

In [ ]:
def set_states(series):
    series_iterator = np.nditer(series, flags=['c_index'])
    X = np.zeros(series.shape, dtype=np.int8)
    counting = False
    for x in series_iterator:
        i = series_iterator.index
        if x > start and not counting:
            X[i] = 1
            counting = True
        elif x < end and counting:
            X[i] = -1
            counting = False
    return X

In [ ]:
start_time = time.time()
gf_states = gf2.apply(set_states, raw=True)
print(f'Execution time: {time.time() - start_time}')
gf_states

In [ ]:
sns.heatmap(gf_states.T, cmap='coolwarm')

In [ ]:
def select_date_ranges(series):
    date_ranges = []
    counting = False
    for i, x in series.items():
        if x > start and not counting:
            start_index = i
            counting = True
        elif x < end and counting:
            end_index = i
            date_ranges.append((start_index, end_index))
            counting = False
    return date_ranges

date_ranges = gf2.apply(select_date_ranges)
date_ranges

In [ ]:
start_time = time.time()
stabilvol = []
for stock, series in gf2.items():
    for interval in date_ranges[stock]:
        chunck = series.loc[interval[0]: interval[1]]
        volatility = chunck.std()
        fht = len(chunck)
        stabilvol.append((volatility, fht))
end_time = time.time()
print(f'Execution time: {end_time - start_time}')